In [1]:
import os
os.chdir('/content/drive/MyDrive/yeonjun/공부/RecSys/intro_to_recsys/data')

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from datetime import datetime

import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [3]:
train = np.load('./ml-100k/ml_100k_train.npy')
test = np.load('./ml-100k/ml_100k_test.npy')

train = (train > 0).astype(float)
test = (test > 0).astype(float)

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
class Config:
    learning_rate = 0.001
    weight_decay = 0.01
    early_stopping_round = 0
    epochs = 20
    seed = 1995
    embed_dim = 50
    hidden_dim = [64, 32, 16]

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batch_size = 128

config = Config()

In [6]:
class TrainDataset(Dataset):
    def __init__(self, data, neg_data_per_pos_data):
        super(TestDataset).__init__()
        self.M = data.shape[0]
        self.N = data.shape[1]
        self.data = data
        idx_mat = np.arange(self.M * self.N).reshape(self.M, self.N)
        pos_n = np.sum(data, dtype=np.int16)
        
        neg_idx = idx_mat[data == 0]
        pos_idx = idx_mat[data == 1]

        neg_sampled_idx = np.random.choice(neg_idx, pos_n*neg_data_per_pos_data, replace=False)
        self.total_rate = np.sort(np.union1d(pos_idx, neg_sampled_idx))

    def __len__(self):
        return len(self.total_rate)
        
    def __getitem__(self, i):
        idx = self.total_rate[i]
        u = int(idx // self.N)
        i = int(idx % self.M)
        r = self.data[u, i]

        return (u, i, r)

class TestDataset(Dataset):
    def __init__(self, data):
        super(TestDataset).__init__()
        self.M = data.shape[0]
        self.N = data.shape[1]
        self.data = data

    def __len__(self):
        return self.M * self.N
        
    def __getitem__(self, idx):
        u = int(idx // self.N)
        i = int(idx % self.M)
        r = self.data[u, i]
        
        return (u, i, r)

In [9]:
class MLP(nn.Module):
    def __init__(self, user_dim, item_dim, embed_dim, hidden_dim):
        super(MLP, self).__init__()
        self.embed_dim = embed_dim

        self.user_embed_mlp = nn.Embedding(user_dim, embed_dim)
        self.item_embed_mlp = nn.Embedding(item_dim, embed_dim)
        self.mlp_1 = nn.Linear(embed_dim*2, hidden_dim[0])
        self.mlp_2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.mlp_3 = nn.Linear(hidden_dim[1], hidden_dim[2])

        self.out = nn.Linear(hidden_dim[2], 1)
        self.sig = nn.Sigmoid()

        self._init_weight_()

    def _init_weight_(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.Embedding):
                m.weight.data.normal_(0, 0.02)

    def forward(self, user_idx, item_idx):
        p = self.user_embed_mlp(user_idx)
        q = self.item_embed_mlp(item_idx)
        interact = torch.cat((p, q), axis=1)
        # print(interact.shape) # (batch_size, embed_dim*2)
        
        out = F.relu(self.mlp_1(interact))
        out = F.relu(self.mlp_2(out))
        out = F.relu(self.mlp_3(out))

        out = self.out(out)
        out = self.sig(out)

        return out


In [10]:
seed_everything(config.seed)

train_data = TrainDataset(train, neg_data_per_pos_data=4)
test_data = TestDataset(test)

train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=config.batch_size*100, shuffle=False)

model = MLP(train.shape[0], train.shape[1], config.embed_dim, config.hidden_dim)
model.to(config.device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
loss_fn = nn.BCEWithLogitsLoss()

start = datetime.now()
history = defaultdict(list)
history['best_loss'] = np.inf
for epoch in range(config.epochs):
    
    model.train()
    losses = 0
    for batch_data in train_loader:
        user = batch_data[0].to(config.device, dtype=torch.long)
        item = batch_data[1].to(config.device, dtype=torch.long)
        rate = batch_data[2].to(config.device, dtype=torch.float)

        optimizer.zero_grad()
        
        pred = model(user, item)
        loss = loss_fn(pred, rate.unsqueeze(-1))
        loss.backward()
        optimizer.step()

        losses += loss.item()
    losses /= len(train_loader) 
    history['train_losses'].append(losses)

    losses_val = 0
    for bacth_data in test_loader:
        user = batch_data[0].to(config.device, dtype=torch.long)
        item = batch_data[1].to(config.device, dtype=torch.long)
        rate = batch_data[2].to(config.device, dtype=torch.float)

        with torch.no_grad():

            pred = model(user, item)
            loss = loss_fn(pred, rate.unsqueeze(-1))
            losses_val += loss.item()

    losses_val /= len(test_loader)
    
    print(f'EPOCH {epoch+1} TRAIN LogLoss : {losses:.6f}, TEST LogLoss : {losses_val:.6f}')

    history['val_losses'].append(losses_val)
    if history['best_loss'] > losses_val:
        history['best_loss'] = losses_val
        torch.save(model.state_dict(), f'../paper_review/2. Implicit feedback/Neural network based/pretrained/mlp_pretrained.pth')
        print('The Model Saving...')
    # if epoch==0 or (epoch + 1) % 10 == 0 or epoch == config.epochs:

end = datetime.now()
print(f'Training takes time {end-start}')

EPOCH 1 TRAIN LogLoss : 0.718237, TEST LogLoss : 0.698769
The Model Saving...
EPOCH 2 TRAIN LogLoss : 0.698713, TEST LogLoss : 0.698281
The Model Saving...
EPOCH 3 TRAIN LogLoss : 0.698901, TEST LogLoss : 0.697653
The Model Saving...
EPOCH 4 TRAIN LogLoss : 0.699050, TEST LogLoss : 0.698855
EPOCH 5 TRAIN LogLoss : 0.699125, TEST LogLoss : 0.700082
EPOCH 6 TRAIN LogLoss : 0.699138, TEST LogLoss : 0.699850
EPOCH 7 TRAIN LogLoss : 0.699141, TEST LogLoss : 0.699043
EPOCH 8 TRAIN LogLoss : 0.699140, TEST LogLoss : 0.698570
EPOCH 9 TRAIN LogLoss : 0.699139, TEST LogLoss : 0.699690
EPOCH 10 TRAIN LogLoss : 0.699144, TEST LogLoss : 0.699253
EPOCH 11 TRAIN LogLoss : 0.699141, TEST LogLoss : 0.699684
EPOCH 12 TRAIN LogLoss : 0.699140, TEST LogLoss : 0.697808
EPOCH 13 TRAIN LogLoss : 0.699143, TEST LogLoss : 0.698778
EPOCH 14 TRAIN LogLoss : 0.699135, TEST LogLoss : 0.699832
EPOCH 15 TRAIN LogLoss : 0.699145, TEST LogLoss : 0.699287
EPOCH 16 TRAIN LogLoss : 0.699139, TEST LogLoss : 0.698987
EPOCH

In [10]:
history['best_loss']